In [4]:
import pandas as pd
import os 
import time
import datetime
import functools
from methods.runner import Runner

In [8]:
folder = "data"

files = sorted(os.listdir(folder))

runner = Runner(models = None)
min_validation_size = 8
num_steps = 4
validation_split = 0.05
min_size = min_validation_size * (num_steps - 1) / validation_split

merged_df = None

for i, file in enumerate(files):
	df = pd.read_csv(os.path.join(folder, file), index_col='Period').sort_index()

	if df.shape[0] < min_size:
		continue

	df = runner.get_df(df)
	df = df.pct_change()
	df.dropna(inplace=True)

	if merged_df is None:
		merged_df = df
	else:
		merged_df = pd.concat([merged_df, df])

print(merged_df.shape)


(2032, 1)


In [9]:
merged_df.head()

,value
Period,
201002,0.251804
201003,0.112559
201004,0.198589
201005,-0.107327
201006,0.073276


In [31]:
import numpy as np

delta = 0.060

def get_variation_count(df, delta):
	positive = df[df["value"] > delta]
	negative = df[df["value"] < -delta]
	neutral = df[(df["value"] <= delta) & (df["value"] >= -delta)]

	return positive.shape[0], negative.shape[0], neutral.shape[0]

def get_variance(df, delta):
	positive, negative, neutral = get_variation_count(df, delta)
	min_val = min(positive, negative, neutral)

	variances = [positive, negative, neutral]

	return positive - min_val + negative - min_val + neutral - min_val, variances

min_variance = None
min_delta_variance = None
min_variances = None

delta_var = 0.00005
qtd = int(1/delta_var)

deltas = [x * delta_var for x in range(qtd+1)]

for delta in deltas:
	variance, variances = get_variance(merged_df, delta)

	if min_variance is None:
		min_variance = variance
		min_delta_variance = delta
		min_variances = variances
	
	elif variance < min_variance:
		min_variance = variance
		min_delta_variance = delta
		min_variances = variances

print(min_variance, min_delta_variance, min_variances)

# print(get_variation_count(merged_df, delta))

103 0.055600000000000004 [746, 643, 643]
